In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
%load_ext google.cloud.bigquery
from google.cloud import bigquery

client=bigquery.Client()
project_id = "som-nero-phi-jonc101"

/Users/sandychen/Desktop/Healthrex_workspace/scripts/Blood_Culture_Stewardship/blood_culture_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sandychen/Desktop/Healthrex_workspace/scripts/Blood_Culture_Stewardship/blood_culture_env/lib/python3.9/site-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


In [2]:
# demo
table_id_final_base_demo = f"{project_id}.blood_culture_stewardship_peds_sandy_2024.final_base_bmi_age_race_gender_adi_temp_peds"
final_base_demo = client.query(f"SELECT * FROM `{table_id_final_base_demo}`").to_dataframe()
# labs
table_id_final_base_labs_lda_ua = f"{project_id}.blood_culture_stewardship_peds_sandy_2024.final_base_labs_without_cpr_or_pct_ua_lda_peds"
final_base_labs_ua_lda= client.query(f"SELECT * FROM `{table_id_final_base_labs_lda_ua}`").to_dataframe()
# vitals
table_id_final_base_vitals = f"{project_id}.blood_culture_stewardship_peds_sandy_2024.final_base_vitals_peds"
final_base_vitals= client.query(f"SELECT * FROM `{table_id_final_base_vitals}`").to_dataframe()

table_id_final_labs_cr_only = f"{project_id}.blood_culture_stewardship_peds_sandy_2024.final_base_labs_cr_only_peds"
final_labs_cr_only = client.query(f"SELECT * FROM `{table_id_final_labs_cr_only}`").to_dataframe()



In [3]:
# Alternative: Using functools.reduce to merge multiple dataframes at once
from functools import reduce

# Define merge keys
key_cols = ["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "blood_culture_order_datetime_utc"]

# List of dataframes to merge
dfs_to_merge = [final_base_demo, final_base_labs_ua_lda, final_base_vitals, final_labs_cr_only]

# Merge all dataframes
all_features_df = reduce(
    lambda left, right: pd.merge(left, right, on=key_cols, how="inner"),
    dfs_to_merge
)

print(f"Shape of merged features: {all_features_df.shape}")
print(f"Number of rows: {len(all_features_df)}")
all_features_df.head()


Shape of merged features: (27515, 90)
Number of rows: 27515


,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,order_year,age_days,age_years,gender,race,bmi,source,positive_blood_culture,zip_clean,adi_score,adi_imputed_flag,min_wbc,max_wbc,avg_wbc,median_wbc,min_neutrophils,max_neutrophils,avg_neutrophils,median_neutrophils,min_anc,max_anc,avg_anc,median_anc,min_lymphocytes,max_lymphocytes,avg_lymphocytes,median_lymphocytes,min_alc,max_alc,avg_alc,median_alc,min_hgb,max_hgb,avg_hgb,median_hgb,min_plt,max_plt,avg_plt,median_plt,min_glucose,max_glucose,avg_glucose,median_glucose,min_lactate,max_lactate,avg_lactate,median_lactate,Leukocyte_Esterase,WBC_urine,Bacteria_urine,Nitrite_urine,has_any_line,temp_min_c,temp_avg_c,temp_max_c,temp_median_c,temp_mode_c,resp_min,resp_avg,resp_max,resp_median,resp_mode,hr_min,hr_avg,hr_max,hr_median,hr_mode,sysbp_min,sysbp_avg,sysbp_max,sysbp_median,sysbp_mode,diabp_min,diabp_avg,diabp_max,diabp_median,diabp_mode,spo2_min,spo2_avg,spo2_max,spo2_median,spo2_mode,min_cr,max_cr,avg_cr,median_cr
0,JC1000964,131181803241,489700509,2016-03-09 05:24:00,2016,4254,11.654795,Male,White,16.11,shc,0,94087,1.291797,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,0,36.611111,36.972222,37.388889,36.722222,36.6,18.0,18.750000,20.0,18.0,18.0,80.0,85.250000,90.0,84.0,80.0,105.0,109.0,117.0,105.0,105.0,57.0,61.250000,71.0,58.0,57.0,97.0,98.250000,99.0,98.0,99.0,NaN,NaN,NaN,NaN
1,JC1000964,313661322,717728953,2016-03-09 07:02:00,2016,4254,11.654795,Male,White,16.11,lpch,0,94087,1.291797,1,13.0,13.0,13.0,13.0,74.3,74.3,74.3,74.3,9.67,9.67,9.67,9.67,16.3,16.3,16.3,16.3,2.13,2.13,2.13,2.13,11.3,11.3,11.3,11.3,267.0,267.0,267.0,267.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,0,36.611111,36.972222,37.388889,36.722222,36.6,18.0,18.750000,20.0,18.0,18.0,80.0,84.666667,90.0,84.0,84.0,104.0,108.0,117.0,105.0,105.0,57.0,62.400000,71.0,59.0,57.0,95.0,97.500000,99.0,97.0,97.0,NaN,NaN,NaN,NaN
2,JC1001312,131092528115,467723412,2015-06-19 17:18:00,2015,3983,10.912329,Male,Other,27.27,shc,0,94085,1.815900,1,25.9,25.9,25.9,25.9,82.9,82.9,82.9,82.9,21.48,21.48,21.48,21.48,12.8,12.8,12.8,12.8,3.32,3.32,3.32,3.32,16.6,16.6,16.6,16.6,444.0,444.0,444.0,444.0,448.0,601.0,524.5,448.0,4.32,4.32,4.32,4.32,None,None,None,None,0,36.111111,36.416667,36.722222,36.111111,36.1,20.0,23.333333,26.0,24.0,20.0,101.0,108.200000,120.0,102.0,102.0,115.0,119.0,124.0,118.0,115.0,67.0,70.333333,75.0,69.0,67.0,98.0,98.333333,99.0,98.0,98.0,1.1,1.1,1.1,1.1
3,JC1001312,311762468,711125650,2015-06-19 18:32:00,2015,3983,10.912329,Male,Other,27.27,lpch,0,94085,1.815900,1,25.9,25.9,25.9,25.9,82.9,82.9,82.9,82.9,21.48,21.48,21.48,21.48,12.8,12.8,12.8,12.8,3.32,3.32,3.32,3.32,16.6,16.6,16.6,16.6,444.0,444.0,444.0,444.0,601.0,601.0,601.0,601.0,4.32,4.32,4.32,4.32,None,None,None,None,0,36.111111,36.416667,36.722222,36.111111,36.1,20.0,23.000000,26.0,23.0,24.0,101.0,118.272727,130.0,122.0,102.0,115.0,123.5,134.0,120.0,115.0,62.0,70.666667,76.0,69.0,75.0,96.0,97.166667,99.0,96.0,96.0,1.1,1.1,1.1,1.1
4,JC1001474,131270266094,615238072,2019-05-21 14:13:00,2019,5432,14.882192,Female,Other,22.32,shc,0,95667,25.333756,1,18.5,18.5,18.5,18.5,89.6,89.6,89.6,89.6,16.63,16.63,16.63,16.63,6.1,6.1,6.1,6.1,1.13,1.13,1.13,1.13,13.6,13.6,13.6,13.6,295.0,295.0,295.0,295.0,260.0,287.0,273.5,260.0,6.12,6.12,6.12,6.12,NEGATIVE,NEGATIVE,POSITIVE,NEGATIVE,0,38.111111,38.111111,38.111111,38.111111,38.1,16.0,25.000000,39.0,25.0,16.0,125.0,132.200000,138.0,135.0,125.0,88.0,88.0,88.0,88.0,88.0,45.0,45.000000,45.0,45.0,45.0,95.0,96.600000,98.0,96.0,96.0,0.8,0.8,0.8,0.8


In [32]:
for index, i in enumerate(all_features_df.isnull().mean()*100):
    print(f"{all_features_df.columns[index]}: {i}%")

anon_id: 0.0%
pat_enc_csn_id_coded: 0.0%
order_proc_id_coded: 0.0%
blood_culture_order_datetime_utc: 0.0%
order_year: 0.0%
age_days: 0.0%
age_years: 0.0%
gender: 0.0%
race: 0.0%
bmi: 7.392331455569689%
source: 0.0%
positive_blood_culture: 0.0%
zip_clean: 6.647283300018173%
adi_score: 0.0%
adi_imputed_flag: 0.0%
min_wbc: 8.108304561148465%
max_wbc: 8.108304561148465%
avg_wbc: 8.108304561148465%
median_wbc: 8.108304561148465%
min_neutrophils: 51.266581864437576%
max_neutrophils: 51.266581864437576%
avg_neutrophils: 51.266581864437576%
median_neutrophils: 51.266581864437576%
min_anc: 24.641104851898966%
max_anc: 24.641104851898966%
avg_anc: 24.641104851898966%
median_anc: 24.641104851898966%
min_lymphocytes: 18.026530983100127%
max_lymphocytes: 18.026530983100127%
avg_lymphocytes: 18.026530983100127%
median_lymphocytes: 18.026530983100127%
min_alc: 18.826094857350537%
max_alc: 18.826094857350537%
avg_alc: 18.826094857350537%
median_alc: 18.826094857350537%
min_hgb: 7.784844630201708%
max_

In [22]:
all_features_df["age"] = np.floor(all_features_df["age_years"])

In [20]:
all_features_df["age_years"]

0        11.654795
1        11.654795
2        10.912329
3        10.912329
4        14.882192
           ...    
27510    11.819178
27511    14.657534
27512    14.657534
27513    17.558904
27514    17.558904
Name: age_years, Length: 27515, dtype: float64

## data split

### split into train, val, test (with ~70%,~15%,~15%)
### use longitudinal split
### train: start - 2023 (Exclusive)
### val: 2023-2024 (exclusive)
### test: 2024-now (inclusive)
### the prevalence is about 5% for all data split

In [23]:

print(f"total number of rows: {all_features_df.shape[0]}")
train_data = all_features_df[all_features_df['order_year'] < 2023]
val_data = all_features_df[(all_features_df['order_year'] >= 2023) & (all_features_df['order_year'] < 2024)]
test_data = all_features_df[all_features_df['order_year'] >= 2024]

print(f"Train data shape: {train_data.shape} and is {train_data.shape[0] / all_features_df.shape[0] * 100}% of all data")
print(f"Val data shape: {val_data.shape} and is {val_data.shape[0] / all_features_df.shape[0] * 100}% of all data")
print(f"Test data shape: {test_data.shape} and is {test_data.shape[0] / all_features_df.shape[0] * 100}% of all data")

total number of rows: 27515
Train data shape: (20263, 91) and is 73.64346719970925% of all data
Val data shape: (3183, 91) and is 11.56823550790478% of all data
Test data shape: (4069, 91) and is 14.788297292385971% of all data


In [24]:
# prevalence of blood culture positive
print(f"prevalence of blood culture positive in all data: {all_features_df['positive_blood_culture'].mean() *100}%")
print(f"prevalence of blood culture positive in train data: {train_data['positive_blood_culture'].mean() *100}%")
print(f"prevalence of blood culture positive in val data: {val_data['positive_blood_culture'].mean() *100}%")
print(f"prevalence of blood culture positive in test data: {test_data['positive_blood_culture'].mean() *100}%")

prevalence of blood culture positive in all data: 2.7584953661639107%
prevalence of blood culture positive in train data: 3.188076790208755%
prevalence of blood culture positive in val data: 1.3509267986176563%
prevalence of blood culture positive in test data: 1.7203244040304742%


### process demo data
##### drop unknown gender and impute BMI by age and gender

In [26]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import sklearn

# ===============================
# 0) Helpers
# ===============================
def fill_demo_unknown(df):
    out = df.copy()
    if "gender" in out.columns:
        out["gender"] = out["gender"].fillna("Unknown")
    if "race" in out.columns:
        out["race"] = out["race"].fillna("Unknown")
    return out

def make_age_group(df, age_col="age"):
    bins   = [18, 40, 65, 200]  # adult-only
    labels = ["young_adult", "middle_age", "older_adult"]
    out = df.copy()
    out["age_group"] = pd.cut(out[age_col], bins=bins, labels=labels, right=False)
    return out

def fit_group_medians(train_df, group_cols, numeric_cols):
    gm = (train_df.groupby(group_cols, dropna=False)[numeric_cols]
                  .median()
                  .reset_index())
    global_medians = train_df[numeric_cols].median()
    return gm, global_medians

def apply_stratified_impute(df, group_cols, group_medians, global_medians,
                            numeric_cols, ua_defaults):
    out = df.copy()
    out = out.merge(group_medians, on=group_cols, how="left", suffixes=("", "__grpmed"))

    # numeric columns: per-group median -> global median
    for col in numeric_cols:
        if col not in out.columns:
            continue
        out[f"{col}_missing_flag"] = out[col].isna().astype(int)
        grp_col = f"{col}__grpmed"
        out[col] = out[col].fillna(out.get(grp_col))
        if out[col].isna().any():
            out[col] = out[col].fillna(global_medians.get(col))
        if grp_col in out.columns:
            out.drop(columns=[grp_col], inplace=True)

    # UA + binary defaults
    for col, default_val in ua_defaults.items():
        if col in out.columns:
            out[f"{col}_missing_flag"] = out[col].isna().astype(int)
            out[col] = out[col].fillna(default_val)
            if col == "has_any_line":
                try:
                    out[col] = out[col].astype("Int64")
                except Exception:
                    pass
    return out

def check_missing(df, cols, name):
    miss = df[cols].isna().mean().sort_values(ascending=False).head(20)
    print(f"[{name}] top remaining missing (should be 0):\n{miss}\n")

# ===============================
# 1) Fill demo Unknown + make age_group
# ===============================
for name in ("train_data", "val_data", "test_data"):
    locals()[name] = fill_demo_unknown(locals()[name])
    locals()[name] = make_age_group(locals()[name], age_col="age")

# ===============================
# 2) Feature selection (ALL vitals + ALL lab aggregates)
# ===============================
label_key = "positive_blood_culture"
demo_cols = ["gender", "race", "age", "bmi", "adi_score"]

vital_prefixes = ("temp_", "hr_", "sysbp_", "diabp_", "spo2_", "resp_")
lab_bases      = ("wbc","neutrophils","anc","lymphocytes","alc","hgb","plt","glucose","lactate", "cr")
lab_agg_prefix = ("min_", "max_", "avg_", "median_", "mode_")  # mode_* may not exist for all

ua_cols_all    = ["Leukocyte_Esterase","WBC_urine","Bacteria_urine","Nitrite_urine","has_any_line"]

all_cols = set(train_data.columns)

# vitals present
vital_cols = [c for c in all_cols if c.startswith(vital_prefixes)]

# lab aggregates present (min/max/avg/median/mode for chosen lab families)
lab_agg_cols = [
    c for c in all_cols
    if any(c.startswith(pref) for pref in lab_agg_prefix)
    and any(c.endswith(f"_{base}") for base in lab_bases)
]

# UA present
ua_cols_present = [c for c in ua_cols_all if c in all_cols]

# Final feature list used in modeling
selected_features = [c for c in (demo_cols + vital_cols + lab_agg_cols + ua_cols_present) if c in all_cols]

# Numeric targets for stratified imputation (vitals + all lab aggs + numeric demos)
numeric_targets = [
    c for c in (list(vital_cols) + list(lab_agg_cols) + ["age","bmi","adi_score"])
    if c in train_data.columns and pd.api.types.is_numeric_dtype(train_data[c])
]

# UA defaults for missing (categorical dipsticks as NEGATIVE, line as 0)
ua_defaults = {
    "Leukocyte_Esterase": "NEGATIVE",
    "WBC_urine": "NEGATIVE",
    "Bacteria_urine": "NEGATIVE",
    "Nitrite_urine": "NEGATIVE",
    "has_any_line": 0,   # keep as binary
}

# ===============================
# 3) Train-only stratified imputation by (gender × age_group)
# ===============================
group_cols = ["gender", "age_group"]
group_meds, global_meds = fit_group_medians(train_data, group_cols, numeric_targets)

train_imp = apply_stratified_impute(train_data, group_cols, group_meds, global_meds,
                                    numeric_targets, ua_defaults)
val_imp   = apply_stratified_impute(val_data,   group_cols, group_meds, global_meds,
                                    numeric_targets, ua_defaults)
test_imp  = apply_stratified_impute(test_data,  group_cols, group_meds, global_meds,
                                    numeric_targets, ua_defaults)

# Sanity checks (only on columns we actually use)
check_cols = [c for c in (numeric_targets + ua_cols_present) if c in train_imp.columns]
check_missing(train_imp, check_cols, "train")
check_missing(val_imp,   check_cols, "val")
check_missing(test_imp,  check_cols, "test")

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_98021/51115027.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  gm = (train_df.groupby(group_cols, dropna=False)[numeric_cols]
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_98021/51115027.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{col}_missing_flag"] = out[col].isna().astype(int)
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_98021/51115027.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, wh

[train] top remaining missing (should be 0):
spo2_avg        0.0
hr_max          0.0
spo2_min        0.0
temp_mode_c     0.0
diabp_median    0.0
resp_max        0.0
spo2_median     0.0
diabp_max       0.0
resp_median     0.0
sysbp_avg       0.0
temp_avg_c      0.0
hr_median       0.0
resp_mode       0.0
diabp_min       0.0
diabp_avg       0.0
spo2_mode       0.0
sysbp_median    0.0
resp_min        0.0
resp_avg        0.0
spo2_max        0.0
dtype: float64

[val] top remaining missing (should be 0):
spo2_avg        0.0
hr_max          0.0
spo2_min        0.0
temp_mode_c     0.0
diabp_median    0.0
resp_max        0.0
spo2_median     0.0
diabp_max       0.0
resp_median     0.0
sysbp_avg       0.0
temp_avg_c      0.0
hr_median       0.0
resp_mode       0.0
diabp_min       0.0
diabp_avg       0.0
spo2_mode       0.0
sysbp_median    0.0
resp_min        0.0
resp_avg        0.0
spo2_max        0.0
dtype: float64

[test] top remaining missing (should be 0):
spo2_avg        0.0
hr_max          

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_98021/51115027.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{col}_missing_flag"] = out[col].isna().astype(int)
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_98021/51115027.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{col}_missing_flag"] = out[col].isna().astype(int)
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_98021/51115027.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of ca

In [27]:
table_id_all_features_imputed_train= f"{project_id}.blood_culture_stewardship_peds_sandy_2024.final_base_features_imputed_train_peds"
table_id_all_features_imputed_val= f"{project_id}.blood_culture_stewardship_peds_sandy_2024.final_base_features_imputed_val_peds"
table_id_all_features_imputed_test= f"{project_id}.blood_culture_stewardship_peds_sandy_2024.final_base_features_imputed_test_peds"

# Upload the DataFrame to BigQuery
train_imp.to_gbq(
    destination_table=table_id_all_features_imputed_train,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)
val_imp.to_gbq(
    destination_table=table_id_all_features_imputed_val,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)
test_imp.to_gbq(
    destination_table=table_id_all_features_imputed_test,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)
print(f"Uploaded train_imp to {table_id_all_features_imputed_train}")
print(f"Uploaded val_imp to {table_id_all_features_imputed_val}")
print(f"Uploaded test_imp to {table_id_all_features_imputed_test}")


/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_98021/1539210140.py:6: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  train_imp.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 1466.54it/s]
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_98021/1539210140.py:11: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  val_imp.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 880.42it/s]
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_98021/1539210140.py:16: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  test_imp.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 15592.2

Uploaded train_imp to som-nero-phi-jonc101.blood_culture_stewardship_peds_sandy_2024.final_base_features_imputed_train_peds
Uploaded val_imp to som-nero-phi-jonc101.blood_culture_stewardship_peds_sandy_2024.final_base_features_imputed_val_peds
Uploaded test_imp to som-nero-phi-jonc101.blood_culture_stewardship_peds_sandy_2024.final_base_features_imputed_test_peds


### imputed train,val test data uploaded to bigquery 
##### Uploaded train_imp to som-nero-phi-jonc101.blood_culture_stewardship_peds_sandy_2024.final_base_features_imputed_train_peds
##### Uploaded val_imp to som-nero-phi-jonc101.blood_culture_stewardship_peds_sandy_2024.final_base_features_imputed_val_peds
##### Uploaded test_imp to som-nero-phi-jonc101.blood_culture_stewardship_peds_sandy_2024.final_base_features_imputed_test_peds